### Word Sense Disambiguity

Given the following sentences:

    The agent will book the to the show for the entire family.
    But you can generally book tickets online.
    When you book tickets online they provide you with a book of stamps
    
If you could see the above sentences the word book is used in different context. In first two sentences the word book(verb) refers to the meaning 'reserve' while in the second portion of the third sentence book(noun) refers to a physical entity.

## Part - 1

    Use the Lesk Module to find the similar words of the word *book* using the above sentences. Record your observations.
    
## Part - 2

Tag sentences using Brill Tagger.

### Brill Tagger

The BrillTagger class is a **transformation-based tagger**. The BrillTagger class uses a series
of rules to correct the results of an initial tagger. These rules are scored based on how many
errors they correct minus the number of new errors they produce.

The idea is simple Brill Tagger tries to correct the mistake made by the inital tagger. Brill tagger inputs an initial tagger and the templates which autmatically tells to create new rules based on the Training Set.

**Recommended Steps:**

1. Initially tag the sentence using POS Tagger. Then observe the POS tags for the word book in different context
2. Then create a tagged_sentence using the POS Tagger correcting it with the mistakes it made.
3. Now create a Brill Tagger using an initial tagger (POS) and pass templates(rules) to it.
4. Train the Brill Tagger using the Tagged Sentence
5. Test the Brill Tagger on the following sentences:
       > "I bought this book from Kerala"
       > "He will book tickets to Kerala"
       
## Part - 3

    Perform Part-1 again but passing the POS tags produced by the Brill Tagger.
    

In [1]:
import nltk
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import state_union
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import brown
from nltk.tag import pos_tag, map_tag

# Regex tagger
from nltk.tag import RegexpTagger
#from nltk.tag import UnigramTagger

#brill tagger
from nltk.tag.brill import *
from nltk.tag import brill
from nltk import brill_trainer
#from nltk.tag.simplify import simplify_wsj_tag

In [2]:
sent_1 = 'The agent will book the to the show for the entire family.'
sent_1 = word_tokenize(sent_1)
sent_2 = 'But you can generally book tickets online.'
sent_2 = word_tokenize(sent_2)
sent_3 = 'When you book tickets online they provide you with a book of stamps.'
sent_3 = word_tokenize(sent_3)


# Part - 1 #

In [3]:
from nltk.wsd import lesk

ambiguous = 'book'

# Using lesk module to find synset

m1 = lesk(sent_1, ambiguous)
m2 = lesk(sent_2, ambiguous)
m3 = lesk(sent_3, ambiguous)
print (m1)
print (m2)
print (m3)

Synset('koran.n.01')
Synset('script.n.01')
Synset('book.n.11')


# Part - 2 #

In [36]:
sent1 = "I bought this book from Kerala"
sent2 = "He will book tickets to Kerala"

token_1 = word_tokenize(sent1)
token_2 = word_tokenize(sent2)

def tok(sen):
    tagged = []
    words = nltk.word_tokenize(sen)
    tagged.append(nltk.pos_tag(words))
    return tagged




####  POS tag output, book is treated as a noun ####

In [37]:
token_sent1 = tok(sent1)
token_sent2 = tok(sent2)
print (token_sent1)
print (token_sent2)

[[('I', 'PRP'), ('bought', 'VBD'), ('this', 'DT'), ('book', 'NN'), ('from', 'IN'), ('Kerala', 'NNP')]]
[[('He', 'PRP'), ('will', 'MD'), ('book', 'NN'), ('tickets', 'NNS'), ('to', 'TO'), ('Kerala', 'VB')]]


####  Corrected POS tag ####

In [38]:
corrected_pos = [[('I', 'PRP'),
      ('will', 'VB'), 
      ('book', 'VB' ),
      ('a', 'DT'),
      ('ticket', 'NN'),
      ('today', 'NN' ),
      ('.', 'PUNCT'),
      ('The', 'DT'),
      ('Amazon', 'NN'),
      ('Books', 'NNS'),
      ('Store', 'NN'),
      ('offers', 'NNS'),
      ('you', 'PRP'),
      ('millions', 'NNS'),
      ('of','IN'),
      ('titles', 'NNS'),
      ('.', 'PUNCT'),]] 

#### File with corrected training data ####

In [39]:
from nltk.corpus.reader import TaggedCorpusReader
malay_tagged = TaggedCorpusReader('.', 'train.txt', sep="\\")
p = list(malay_tagged.tagged_sents())


#### Conditional exponential classifier using pickle ####

In [40]:
from nltk.data import load
tagdict = load('taggers/maxent_treebank_pos_tagger/english.pickle')

####  The Brill tagger ####

In [41]:
def train_brill_tag(train_data):
   
    # Templates for checking the pos 
    templates = [
        brill.Template(brill.Pos([-1])),
        brill.Template(brill.Pos([1])),
        brill.Template(brill.Pos([-2])),
        brill.Template(brill.Pos([2])),
        brill.Template(brill.Pos([-2, -1])),
        brill.Template(brill.Pos([1, 2])),
        brill.Template(brill.Pos([-3, -2, -1])),
        brill.Template(brill.Pos([1, 2, 3])),
        brill.Template(brill.Pos([-1]), brill.Pos([1])),
        brill.Template(brill.Word([-1])),
        brill.Template(brill.Word([1])),
        brill.Template(brill.Word([-2])),
        brill.Template(brill.Word([2])),
        brill.Template(brill.Word([-2, -1])),
        brill.Template(brill.Word([1, 2])),
        brill.Template(brill.Word([-3, -2, -1])),
        brill.Template(brill.Word([1, 2, 3])),
        brill.Template(brill.Word([-1]), brill.Word([1]))]
     
        
    trainer = brill_trainer.BrillTaggerTrainer(tagdict, templates = templates, trace = 3)
    brill_tagger = trainer.train(train_data, max_rules = 10)
    return brill_tagger

#### POS using brill tagger ###

We get the rules using tempalated 

In [42]:
mt = train_brill_tag(p)    


TBL train (fast) (seqs: 1; tokens: 58; tpls: 18; min score: 2; min acc: None)
Finding initial useful rules...
    Found 565 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
   5   5   0   0  | .->PUNCT if Pos:NN@[-2,-1]
   4   4   0   1  | NNP->NN if Pos:DT@[-3,-2,-1]
   3   3   0   0  | IN->ADP if Pos:NN@[-1]
   3   3   0   0  | MD->VB if Pos:PRP@[-1]
   2   2   0   0  | VBP->VB if Pos:PRP@[-1]
   2   2   0   0  | DT->DET if Pos:ADP@[2]


sent_1 = 'The agent will book the to the show for the entire family.'

In [43]:

print(mt.tag(sent_1))

[('The', 'DT'), ('agent', 'NN'), ('will', 'MD'), ('book', 'VB'), ('the', 'DT'), ('to', 'TO'), ('the', 'DET'), ('show', 'NN'), ('for', 'ADP'), ('the', 'DT'), ('entire', 'JJ'), ('family', 'NN'), ('.', 'PUNCT')]


sent_2 = 'But you can generally book tickets online.'

In [44]:
print(mt.tag(sent_2))

[('But', 'CC'), ('you', 'PRP'), ('can', 'VB'), ('generally', 'RB'), ('book', 'VB'), ('tickets', 'NNS'), ('online', 'NN'), ('.', 'PUNCT')]


sent_3 = When you book tickets online they provide you with a book of stamps

In [45]:
print(mt.tag(sent_3))

[('When', 'WRB'), ('you', 'PRP'), ('book', 'VB'), ('tickets', 'NNS'), ('online', 'VBP'), ('they', 'PRP'), ('provide', 'VB'), ('you', 'PRP'), ('with', 'IN'), ('a', 'DET'), ('book', 'NN'), ('of', 'ADP'), ('stamps', 'NNS'), ('.', '.')]


"I bought this book from Kerala"

In [46]:
print(mt.tag(token_1))

[('I', 'PRP'), ('bought', 'VBD'), ('this', 'DET'), ('book', 'NN'), ('from', 'ADP'), ('Kerala', 'NN')]


He will book tickets to Kerala

In [49]:
print(mt.tag(token_2))

[('He', 'PRP'), ('will', 'VB'), ('book', 'VB'), ('tickets', 'NNS'), ('to', 'TO'), ('Kerala', 'NNP')]
